In [18]:
from src.search_space.operations import *
from src.search_space.networks import *

In [3]:
# model = Network(3, 12, 1, eval("Genotype(normal=[('none', 0), ('dil_conv_5x5', 0), ('sep_conv_5x5', 0), ('sep_conv_3x3', 1), ('sep_conv_3x3', 1), ('dil_conv_5x5', 2), ('sep_conv_5x5', 3), ('skip_connect', 4)], normal_concat=[2, 3, 4, 5], reduce=[('none', 0), ('dil_conv_5x5', 0), ('sep_conv_5x5', 0), ('sep_conv_3x3', 1), ('sep_conv_3x3', 1), ('dil_conv_5x5', 2), ('sep_conv_5x5', 3), ('skip_connect', 4)], reduce_concat=[2, 3, 4, 5])"))
# test = 1

In [19]:
import os
os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = '1'
import argparse
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from scipy import stats
from src.utils.utilities import *
from src.metrics.swap import SWAP
from src.datasets.utilities import get_datasets
from src.search_space.networks import *

# Settings for console outputs
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

parser = argparse.ArgumentParser()

# general setting
torch.cuda.is_available()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
SEED = 0
REPEATS = 4
INPUT_SAMPLE = 16



input_file = open("input_2.txt","r")
input_dim = 80
inputs = []
num = 0
for input_data in input_file.readlines():
    data = (input_data.split())[2:]
    data_list = []
    for i in range(input_dim):
        data_list_list = []
        for j in range(input_dim):
            data_list_list.append(int(data[i*input_dim + j]))
        data_list.append(data_list_list)

    # the input chennel
    inputs.append([data_list,data_list,data_list])
    num += 1
    if num >= 300:
        break
input_file.close()
inputs = torch.Tensor(inputs)

cuda


In [3]:
import random
from collections import namedtuple

# 定義基因型
Genotype = namedtuple('Genotype', 'normal normal_concat reduce reduce_concat')
PRIMITIVES = [
    'none',
    'max_pool_3x3',
    'avg_pool_3x3',
    'skip_connect',
    'sep_conv_3x3',
    'sep_conv_5x5',
    'dil_conv_3x3',
    'dil_conv_5x5'
]

def random_genotype():

    steps=4
    
    def valid_random_ops(steps):
        ops = []
        for i in range(steps):
            op1 = (random.choice(PRIMITIVES), random.randint(0, i + 1))
            op2 = (random.choice(PRIMITIVES), random.randint(0, i + 1))
            ops.append(op1)
            ops.append(op2)
        return ops

    normal = valid_random_ops(steps)
    reduce = valid_random_ops(steps)
    normal_concat = list(range(2, 2 + steps))
    reduce_concat = list(range(2, 2 + steps))
    
    genotype = Genotype(normal=normal, normal_concat=normal_concat, reduce=reduce, reduce_concat=reduce_concat)
    
    return genotype

def evaluate_fitness(genotype):
    network = Network(3, 12, 1, eval(str(genotype)))
    network = network.to(device)
    swap = SWAP(model=network, inputs=inputs, device=device, seed=SEED)
    swap_score = []
    for _ in range(REPEATS):
        network = network.apply(network_weight_gaussian_init)
        swap.reinit()
        swap_score.append(swap.forward())
        swap.clear()
    return np.mean(swap_score)

def mutate(genotype):
    def mutate_ops(ops):
        i = random.randint(0, 7)
        if random.random() > 0.5:
            ops[i] = (random.choice(PRIMITIVES), ops[i][1])
        else:
            if i < 2:
                state = random.randint(0, 1)
            elif i < 4:
                state = random.randint(0, 2)
            elif i < 6:
                state = random.randint(0, 3)
            else:
                state = random.randint(0, 4)
            ops[i] = (random.choice(PRIMITIVES), state)
        return ops
    
    if random.random() > 0.5:
        return Genotype(
            normal=mutate_ops(genotype.normal),
            normal_concat=genotype.normal_concat,
            reduce=genotype.reduce,
            reduce_concat=genotype.reduce_concat
        )
    else:
        return Genotype(
            normal=genotype.normal,
            normal_concat=genotype.normal_concat,
            reduce=mutate_ops(genotype.reduce),
            reduce_concat=genotype.reduce_concat
        )

def crossover(parent1, parent2):
    def mix_ops(ops1, ops2):
        return [random.choice(pair) for pair in zip(ops1, ops2)]
    
    normal = mix_ops(parent1.normal, parent2.normal)
    reduce = mix_ops(parent1.reduce, parent2.reduce)
    
    return Genotype(normal=normal, normal_concat=parent1.normal_concat, reduce=reduce, reduce_concat=parent1.reduce_concat)

In [17]:
# 初始化種群
population_size = 4
# 遺傳演算法的參數
num_generations = 5
mutation_rate = 0.1
tournament_size = 2

best_fitness = -1
best_genotype = None

population = []
for _ in range(population_size):
    genotype = random_genotype()
    population.append((genotype, evaluate_fitness(genotype)))



# 遺傳演算法迴圈
for generation in range(num_generations):
    # 評估種群中的每個個體
    # fitnesses = [evaluate_fitness(genotype) for genotype in population]
    population = sorted(population, key=lambda x: -x[1])
    for gene in population:
        print(gene,end=" ")
    print("")

    max_fitness = population[0][1]
    # max_index = fitnesses.index(max_fitness)
    if max_fitness > best_fitness:
        best_fitness = max_fitness
        best_genotype = population[0][0]

    # 打印當前世代的最佳適應度
    print(f"Generation {generation}: Best Fitness = {max_fitness} Genotype = {best_genotype}")

    # 選擇父代
    new_population = []
    for _ in range(population_size):
        tournament = random.sample(population, tournament_size)
        tournament = sorted(tournament, key=lambda x: -x[1])
        parent1 = tournament[0][0]
        parent2 = tournament[1][0]
        child = crossover(parent1, parent2)
        if random.uniform(0, 1) < mutation_rate:
            child = mutate(child)
        new_population.append((child, evaluate_fitness(child)))
    new_population = sorted(new_population, key=lambda x: -x[1])
    
    population[-1] = new_population[0]

OutOfMemoryError: CUDA out of memory. Tried to allocate 286.00 MiB. GPU 0 has a total capacity of 2.00 GiB of which 0 bytes is free. Of the allocated memory 866.13 MiB is allocated by PyTorch, and 63.87 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [1]:
import torch
import torch.nn as nn

OPS = {
    'none': lambda C_in, C_out, stride, affine: Zero(C_in, C_out, stride),
    'avg_pool_3x3': lambda C_in, C_out, stride, affine: POOLING(C_in, C_out, stride, 'avg', affine),
    'max_pool_3x3': lambda C_in, C_out, stride, affine: POOLING(C_in, C_out, stride, 'max', affine),
    'skip_connect': lambda C_in, C_out, stride, affine: Identity() if stride == 1 and C_in == C_out else FactorizedReduce(C_in, C_out, stride, affine),
    'sep_conv_3x3': lambda C_in, C_out, stride, affine: SepConv(C_in, C_out, 3, stride, 1, affine),
    'sep_conv_5x5': lambda C_in, C_out, stride, affine: SepConv(C_in, C_out, 5, stride, 2, affine),
    'dil_conv_3x3': lambda C_in, C_out, stride, affine: DilConv(C_in, C_out, 3, stride, 2, 2, affine),
    'dil_conv_5x5': lambda C_in, C_out, stride, affine: DilConv(C_in, C_out, 5, stride, 4, 2, affine),
}


class ReLUConvBN(nn.Module):

    def __init__(self, C_in, C_out, kernel_size, stride, padding, dilation, affine, track_running_stats=True):
        super(ReLUConvBN, self).__init__()
        self.op = nn.Sequential(
            nn.ReLU(inplace=False),
            nn.Conv2d(C_in, C_out, kernel_size, stride=stride, padding=padding, dilation=dilation, bias=False),
            nn.BatchNorm2d(C_out, affine=affine, track_running_stats=track_running_stats)
        )

    def forward(self, x):
        return self.op(x)


class DilConv(nn.Module):

    def __init__(self, C_in, C_out, kernel_size, stride, padding, dilation, affine=True, track_running_stats=True):
        super(DilConv, self).__init__()
        self.op = nn.Sequential(
            nn.ReLU(inplace=False),
            nn.Conv2d(C_in, C_in, kernel_size=kernel_size, stride=stride, padding=padding, dilation=dilation,
                      groups=C_in, bias=False),
            nn.Conv2d(C_in, C_out, kernel_size=1, padding=0, bias=False),
            nn.BatchNorm2d(C_out, affine=affine, track_running_stats=track_running_stats),
        )

    def forward(self, x):
        return self.op(x)


class SepConv(nn.Module):

    def __init__(self, C_in, C_out, kernel_size, stride, padding, affine=True, track_running_stats=True):
        super(SepConv, self).__init__()
        self.op = nn.Sequential(
            nn.ReLU(inplace=False),
            nn.Conv2d(C_in, C_in, kernel_size=kernel_size, stride=stride, padding=padding, groups=C_in, bias=False),
            nn.Conv2d(C_in, C_in, kernel_size=1, padding=0, bias=False),
            nn.BatchNorm2d(C_in, affine=affine, track_running_stats=track_running_stats),
            
            nn.ReLU(inplace=False),
            nn.Conv2d(C_in, C_in, kernel_size=kernel_size, stride=1, padding=padding, groups=C_in, bias=False),
            nn.Conv2d(C_in, C_out, kernel_size=1, padding=0, bias=False),
            nn.BatchNorm2d(C_out, affine=affine, track_running_stats=track_running_stats),
        )

    def forward(self, x):
        return self.op(x)


class Identity(nn.Module):

    def __init__(self):
        super(Identity, self).__init__()

    def forward(self, x):
        return x


class FactorizedReduce(nn.Module):
    def __init__(self, C_in, C_out, stride=2, affine=True, track_running_stats=True):
        super(FactorizedReduce, self).__init__()
        self.stride = stride
        self.C_in   = C_in
        self.C_out  = C_out
        self.relu   = nn.ReLU(inplace=False)
        if stride == 2:
            C_outs = [C_out // 2, C_out - C_out // 2]
            self.convs = nn.ModuleList()
            for i in range(2):
                self.convs.append( nn.Conv2d(C_in, C_outs[i], 1, stride=stride, padding=0, bias=False))
            self.pad = nn.ConstantPad2d((0, 1, 0, 1), 0)
        elif stride == 1:
            self.conv = nn.Conv2d(C_in, C_out, 1, stride=stride, padding=0, bias=False)
        else:
            raise ValueError('Invalid stride : {:}'.format(stride))
        self.bn = nn.BatchNorm2d(C_out, affine=affine, track_running_stats=track_running_stats)

    def forward(self, x):
        if self.stride == 2:
            x = self.relu(x)
            y = self.pad(x)
            out = torch.cat([self.convs[0](x), self.convs[1](y[:, :, 1:, 1:])], dim=1)
        else:
            out = self.conv(x)
        out = self.bn(out)
        return out

    def extra_repr(self):
        return 'C_in={C_in}, C_out={C_out}, stride={stride}'.format(**self.__dict__)


class Zero(nn.Module):

    def __init__(self, C_in, C_out, stride):
        super(Zero, self).__init__()
        self.C_in   = C_in
        self.C_out  = C_out
        self.stride = stride
        self.is_zero = True

    def forward(self, x):
        if self.C_in == self.C_out:
            if self.stride == 1: return x.mul(0.)
            else               : return x[:,:,::self.stride,::self.stride].mul(0.)
        else:
            shape = list(x.shape)
            shape[1] = self.C_out
            zeros = x.new_zeros(shape, dtype=x.dtype, device=x.device)
            return zeros

    def extra_repr(self):
        return 'C_in={C_in}, C_out={C_out}, stride={stride}'.format(**self.__dict__)


class POOLING(nn.Module):

    def __init__(self, C_in, C_out, stride, mode, affine=True, track_running_stats=True):
        super(POOLING, self).__init__()
        if C_in == C_out:
            self.preprocess = None
        else:
            self.preprocess = ReLUConvBN(C_in, C_out, 1, 1, 0, 1, affine, track_running_stats)
        if mode == 'avg'  : self.op = nn.AvgPool2d(3, stride=stride, padding=1, count_include_pad=False)
        elif mode == 'max': self.op = nn.MaxPool2d(3, stride=stride, padding=1)
        else              : raise ValueError('Invalid mode={:} in POOLING'.format(mode))

    def forward(self, inputs):
        if self.preprocess: x = self.preprocess(inputs)
        else              : x = inputs
        return self.op(x)



import torch
import torch.nn as nn
import torch.nn.functional as F
from collections import namedtuple

Genotype = namedtuple('Genotype', 'normal normal_concat reduce reduce_concat')

def drop_path(x, drop_prob):
  if drop_prob > 0.:
    x = nn.functional.dropout(x, p=drop_prob)

  return x

class Cell(nn.Module):

  def __init__(self, genotype, C_prev_prev, C_prev, C, reduction, reduction_prev):
    super(Cell, self).__init__()

    if reduction_prev:
      self.preprocess0 = FactorizedReduce(C_prev_prev, C)
    else:
      self.preprocess0 = ReLUConvBN(C_prev_prev, C, 1, 1, 0, 1, True)
    self.preprocess1 = ReLUConvBN(C_prev, C, 1, 1, 0, 1, True)
    
    if reduction:
        op_names, indices = zip(*genotype.reduce)
        concat = genotype.reduce_concat # 2,3,4,5
    else:
        op_names, indices = zip(*genotype.normal)
        concat = genotype.normal_concat # 2,3,4,5
    self._compile(C, op_names, indices, concat, reduction)

  def _compile(self, C, op_names, indices, concat, reduction):
    assert len(op_names) == len(indices)
    self._steps = len(op_names) // 2 # 4
    self._concat = concat # 2,3,4,5
    self.multiplier = len(concat) # 4
    self._ops = nn.ModuleList()

    for name, index in zip(op_names, indices):
        stride = 2 if reduction and index < 2 else 1
        op = OPS[name](C, C, stride, True)
        self._ops += [op]
    self._indices = indices

  def forward(self, s0, s1, drop_prob):
    s0 = self.preprocess0(s0)
    s1 = self.preprocess1(s1)

    states = [s0, s1]
    for i in range(self._steps):
      # print(f"i: {i}")
      h1 = states[self._indices[2*i]]
      h2 = states[self._indices[2*i+1]]
      op1 = self._ops[2*i]
      op2 = self._ops[2*i+1]
      h1 = op1(h1)
      h2 = op2(h2)
      if self.training and drop_prob > 0.:
        if not isinstance(op1, Identity):
          h1 = drop_path(h1, drop_prob)
        if not isinstance(op2, Identity):
          h2 = drop_path(h2, drop_prob)
      s = h1 + h2
      states += [s]
    return torch.cat([states[i] for i in self._concat], dim=1)

class Network(nn.Module):

    def __init__(self, C, num_classes, layers, genotype):
        self.drop_path_prob = 0.
        super(Network, self).__init__()
        
        self._layers = layers

        C_prev_prev, C_prev, C_curr = C, C, C
        
        self.cells = nn.ModuleList()
        reduction_prev = False

        for i in range(layers):
            if i in [layers // 3, 2 * layers // 3]:
                C_curr *= 2
                reduction = True
            else:
                reduction = False
            cell = Cell(genotype, C_prev_prev, C_prev, C_curr, reduction, reduction_prev)
            reduction_prev = reduction
            self.cells += [cell]
            C_prev_prev, C_prev = C_prev, cell.multiplier * C_curr

        self.global_pooling = nn.AdaptiveAvgPool2d(1)
        self.classifier = nn.Linear(C_prev, num_classes)

    def forward(self, input):
        s0 = s1 = input
        
        for i, cell in enumerate(self.cells):
            s0, s1 = s1, cell(s0, s1, self.drop_path_prob)

        out = self.global_pooling(s1)
        out = out.view(out.size(0), -1)
        logits = self.classifier(out)
        return logits



In [7]:
import wrapped_flappy_bird as game
import tensorflow as tf
import cv2
import numpy as np

ACTIONS = 2 

game_state = game.GameState()


#初始化状态并且预处理图片，把连续的四帧图像作为一个输入（State）
do_nothing = np.zeros(ACTIONS)
do_nothing[0] = 1
s_t, r_0, terminal, _ = game_state.frame_step(do_nothing)
s_t = cv2.cvtColor(cv2.resize(s_t, (80, 80)), cv2.COLOR_BGR2GRAY)

model = Network(3, 2, 1, eval(str("Genotype(normal=[('none', 0), ('dil_conv_5x5', 0), ('sep_conv_5x5', 0), ('sep_conv_3x3', 1), ('sep_conv_3x3', 1), ('dil_conv_5x5', 2), ('sep_conv_5x5', 3), ('skip_connect', 4)], normal_concat=[2, 3, 4, 5], reduce=[('none', 0), ('dil_conv_5x5', 0), ('sep_conv_5x5', 0), ('sep_conv_3x3', 1), ('sep_conv_3x3', 1), ('dil_conv_5x5', 2), ('sep_conv_5x5', 3), ('skip_connect', 4)], reduce_concat=[2, 3, 4, 5])")))

while terminal !=True:
    a_t_to_game = np.zeros([ACTIONS])
    action_index = 0
    s_t = cv2.cvtColor(cv2.resize(s_t, (80, 80)), cv2.COLOR_BGR2GRAY)
    s_t = torch.tensor(s_t, dtype=torch.float32)
    s_t = s_t.permute(2, 0, 1)
    s_t = s_t.unsqueeze(0)

    print(s_t.shape)

    readout_t = model(s_t)
    action_index = np.argmax(readout_t.detach().cpu().numpy())
    a_t_to_game[action_index] = 1

    s_t, r_t, terminal, score = game_state.frame_step(a_t_to_game)
    print("============== score ====================")
    print(score)

(288, 512, 3)
(32, 32, 3)
torch.Size([1, 3, 32, 32])
============== score ====================
0
torch.Size([1, 3, 32, 32])
============== score ====================
0
torch.Size([1, 3, 32, 32])
============== score ====================
0
torch.Size([1, 3, 32, 32])
============== score ====================
0
torch.Size([1, 3, 32, 32])
============== score ====================
0
torch.Size([1, 3, 32, 32])
============== score ====================
0
torch.Size([1, 3, 32, 32])
============== score ====================
0
torch.Size([1, 3, 32, 32])
============== score ====================
0
torch.Size([1, 3, 32, 32])
============== score ====================
0
torch.Size([1, 3, 32, 32])
============== score ====================
0
torch.Size([1, 3, 32, 32])
============== score ====================
0
torch.Size([1, 3, 32, 32])
============== score ====================
0
torch.Size([1, 3, 32, 32])
============== score ====================
0
torch.Size([1, 3, 32, 32])
============== score ===